In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [20]:
from graph_datasets import NLGraph

In [35]:
test_dataset = NLGraph(root="./NLGraph", task="connectivity", split="test")
# test_dataset = NLGraph(root="./NLGraph", task="cycle", split="test")
print(test_dataset[0])

({(3, 7), (5, 7), (0, 2), (0, 5), (1, 6), (0, 8), (2, 5), (1, 3), (2, 8), (6, 8), (5, 6), (3, 6), (0, 1), (0, 7), (1, 2), (2, 7), (1, 5), (1, 8), (6, 7), (3, 5), (3, 8), (5, 8), (0, 3), (0, 6), (2, 3), (1, 7), (2, 6), (7, 8)}, ('Determine if there is a path between two nodes in the graph. Note that (i,j) means that node i and node j are connected with an undirected edge.\nGraph', '\nQ: Is there a path between node 8 and node 2?\nA:'), 'The answer is yes.')


In [22]:
model_path = "/data/locus/project_data/project_data2/dylansam/Mistral-7B-Instruct-v0.2"
# model_path = "/data/locus/project_data/project_data2/dylansam/Mixtral-8x7B-Instruct-v0.1"
# model_path = "/data/datasets/models/huggingface/meta-llama/Llama-2-7b-chat-hf"
# model_path = "/data/datasets/models/huggingface/meta-llama/Llama-2-70b-chat-hf"
# model_path = "/data/locus/project_data/project_data2/dylansam/Llama-2-7b-hf"
base_tokenizer = AutoTokenizer.from_pretrained(model_path)
base_tokenizer.pad_token = base_tokenizer.eos_token

In [23]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:13<00:00,  4.60s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

## Try purely naive tokenization (w LLaMA tokenizer) for classification tasks: connectivity and cycle

In [6]:
import torch
from tqdm import tqdm

In [7]:
print(base_tokenizer.encode("The answer is yes"))

[1, 415, 4372, 349, 5081]


In [18]:
print(test_dataset.string_data[0])
print(test_dataset.labels[0][:-4])

Determine if there is a path between two nodes in the graph. Note that (i,j) means that node i and node j are connected with an undirected edge.
Graph: (0,8) (0,1) (0,6) (0,2) (0,3) (0,7) (0,5) (1,8) (1,6) (1,2) (1,3) (1,7) (1,5) (2,8) (2,6) (2,3) (2,7) (2,5) (3,8) (3,6) (3,7) (3,5) (5,8) (5,6) (5,7) (6,8) (6,7) (7,8)
Q: Is there a path between node 8 and node 2?
A:
The answer is 


In [36]:
zs_accuracy = 0

answers = []
preds = []

for i in tqdm(range(len(test_dataset)), total=len(test_dataset)):
    string_question = test_dataset.string_data[i]
    answer = test_dataset[i][2].lower()
    bool_answer = 1 if "yes" in answer else 0

    # input_prompt = string_question + " " + 
    input_prompt = string_question + " The answer is"
    inputs = base_tokenizer(input_prompt, return_tensors="pt").input_ids
    inputs = inputs.cuda()

    with torch.no_grad():
        logits = model(inputs).logits

    # get index of "yes" or "no" answer
    logits = logits[0, -1, :]
    yes_token = base_tokenizer.encode("yes")[1]
    no_token = base_tokenizer.encode("no")[1]
    prob = torch.stack([logits[no_token], logits[yes_token]])
    prob = torch.nn.functional.softmax(prob, dim=0)
    pred = torch.argmax(prob).item()

    preds.append(pred)
    answers.append(bool_answer)

    if pred == bool_answer:
        zs_accuracy += 1

100%|██████████| 371/371 [00:38<00:00,  9.65it/s]


In [37]:
print("ZS Accuracy, naive string input", zs_accuracy / len(test_dataset))

# connectivity ~65%
# cycle ~50%

ZS Accuracy, naive string input 0.6522911051212938


In [38]:
train_dataset = NLGraph(root="./NLGraph", task="connectivity", split="train")
train_dataset = NLGraph(root="./NLGraph", task="cycle", split="train")
print(train_dataset.string_data[0], train_dataset[0][2])

In an undirected graph, (i,j) means that node i and node j are connected with an undirected edge.
The nodes are numbered from 0 to 33, and the edges are: (24,2) (17,26) (1,8) (30,9) (28,31) (3,31) (25,28) (6,9) (28,4) (0,33) (31,10) (23,32) (12,13) (6,16) (15,14) (24,11) (19,21) (24,20) (12,3) (0,24) (13,22) (19,7) (17,23) (29,18) (28,14) (0,17) (5,27) (26,18) (28,2) (8,30) (16,17) (33,21) (21,5)
Q: Is there a cycle in this graph?
A: No, there is no cycle in this graph.


In [39]:
context_examples = ""
for i in range(2):
    answer = train_dataset[i][2].lower()
    answer = 1 if "yes" in answer else 0
    if answer == 1:
        context_examples += train_dataset.string_data[i] + " The answer is yes."
    else:
        context_examples += train_dataset.string_data[i] + " The answer is no."
    
    context_examples += "\n"

In [40]:
print(context_examples)

In an undirected graph, (i,j) means that node i and node j are connected with an undirected edge.
The nodes are numbered from 0 to 33, and the edges are: (24,2) (17,26) (1,8) (30,9) (28,31) (3,31) (25,28) (6,9) (28,4) (0,33) (31,10) (23,32) (12,13) (6,16) (15,14) (24,11) (19,21) (24,20) (12,3) (0,24) (13,22) (19,7) (17,23) (29,18) (28,14) (0,17) (5,27) (26,18) (28,2) (8,30) (16,17) (33,21) (21,5)
Q: Is there a cycle in this graph?
A: The answer is no.
In an undirected graph, (i,j) means that node i and node j are connected with an undirected edge.
The nodes are numbered from 0 to 24, and the edges are: (22,9) (21,16) (13,7) (18,0) (16,22) (17,7) (3,7) (4,17) (8,3) (2,16) (1,13) (14,9) (24,12) (0,15) (5,3) (10,5) (18,19) (9,23) (5,24) (6,22) (11,12) (12,20) (22,20) (0,3)
Q: Is there a cycle in this graph?
A: The answer is no.



In [41]:
icl_accuracy = 0

answers = []
preds = []

for i in tqdm(range(len(test_dataset)), total=len(test_dataset)):

    string_question = test_dataset.string_data[i]
    answer = test_dataset[i][2].lower()
    bool_answer = 1 if "yes" in answer else 0

    input_prompt = context_examples + string_question + " The answer is"
    inputs = base_tokenizer(input_prompt, return_tensors="pt").input_ids
    inputs = inputs.cuda()

    with torch.no_grad():
        logits = model(inputs).logits

    # get index of "yes" or "no" answer
    logits = logits[0, -1, :]
    yes_token = base_tokenizer.encode("yes")[1]
    no_token = base_tokenizer.encode("no")[1]
    prob = torch.stack([logits[no_token], logits[yes_token]])
    prob = torch.nn.functional.softmax(prob, dim=0)
    pred = torch.argmax(prob).item()

    preds.append(pred)
    answers.append(bool_answer)

    if pred == bool_answer:
        icl_accuracy += 1

  7%|▋         | 26/371 [00:04<01:05,  5.27it/s]


KeyboardInterrupt: 

## Test ZS Ability for Shortest Path

In [11]:
test_dataset = NLGraph(root="./NLGraph", task="shortest_path", split="test", string_only=True)

In [12]:
def convert_string_to_graph(string):
    string = string.split(",")
    
    # parsing nodes
    nodes = string[1]
    nodes = nodes.split(" ")
    node_start = nodes[-3]
    node_end = nodes[-1]
    nodes = list(range(int(node_start), int(node_end) + 1))

    # parsing edges
    edges = set()
    for s in string[2:-2]:
        s = s.split(" ")
        node_0 = int(s[-7])
        node_1 = int(s[-4])
        weight = int(s[-1])
        edges.add((node_0, node_1, weight)) # node 0, node 1, edge weight
    return nodes, edges

def parse_answer(string):
    answer_split = string.split(" ")
    path = answer_split[10]
    path = path.split(",")
    path = [int(p) for p in path]
    weight = int(answer_split[-1])
    return path, weight

In [13]:
graph_string = test_dataset[0][0]
answer_string = test_dataset[0][2]

# print(graph_string)
# print(answer_string)
# print(answer_string[: answer_string.find("is") + 3])
# print(convert_string_to_graph(graph_string))
print(parse_answer(answer_string))

([4, 12, 10, 2], 12)


In [14]:
import re

def parse_sp_output(output):
    '''
    Function to parse LLM output for shortest path
    '''
    
    # Split the output to separate the path from the weight description
    # path_parts = re.split(r' with a weight of |\. The total weight of this path is ', output)
    parts = re.split('weight', output)
    path_string = parts[0]
    weight_string = parts[1]

    # Extract the path as numbers
    path = re.findall(r'\d+', path_string)
    path = list(map(int, path))

    # Extract the weight as an integer if present (take the last one?)
    weight = re.findall(r'\d+', weight_string)
    if weight:
        weight = int(weight[0])

        return path, weight
    else:
        return None, None

# Test the corrected function with the original output examples

# Example usage
outputs = [
    "7 -> 1 -> 2 with a weight of 7.",
    "3-1-5-8 with a weight of 7.",
    "8-9-7-3-6-2. The total weight of this path is 22."
]

for output in outputs:
    path, weight = parse_sp_output(output)
    print(path, weight)

[7, 1, 2] 7
[3, 1, 5, 8] 7
[8, 9, 7, 3, 6, 2] 22


In [15]:
zs_accuracy = 0
count = 0

preds = []
actual_labels = []

for i in tqdm(range(len(test_dataset)), total=len(test_dataset)):

    graph_string = test_dataset[i][0]
    answer_string = test_dataset[i][2]

    nodes, edges = convert_string_to_graph(graph_string)
    path, weight = parse_answer(answer_string)

    input_prompt = graph_string + " " + answer_string[: answer_string.find("is") + 3]
    inputs = base_tokenizer(input_prompt, return_tensors="pt").input_ids
    inputs = inputs.cuda()

    with torch.no_grad():
        
        # generate 20 tokens
        outputs = model.generate(inputs, max_length=inputs.shape[1] + 40, do_sample=False, temperature=0.0)
    
    # show decoded output
    decoded_output = base_tokenizer.decode(outputs[0][inputs.shape[1]:])
    try:
        llm_path, llm_weight = parse_sp_output(decoded_output)
    except:
        llm_path, llm_weight = None, None
    
    # print("actual", path, weight)
    # print("pred", llm_path, llm_weight)

    if llm_path == None:
        print(decoded_output)

    else:
        # break
        if path == llm_path and weight == llm_weight:
            zs_accuracy += 1

    count += 1

    preds.append((llm_path, llm_weight))
    actual_labels.append((path, weight))
    # if count == 10:
        # break

print("ZS Accuracy, shortest path", zs_accuracy / count)

  0%|          | 0/64 [00:00<?, ?it/s]/home/dylansam/miniconda3/envs/tokens/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 1/64 [00:01<01:33,  1.48s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  3%|▎         | 2/64 [00:02<01:18,  1.26s/it]The attention mask and the pad token id w

3 edges long and has the following sequence of nodes: 5 -> 1 -> 3. The weight of this path is the sum of the weights of its edges, which is 4 +


 20%|██        | 13/64 [00:17<01:09,  1.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


6 units long and goes through nodes 0, 3, and 6.
Explanation:
To find the shortest path from node 5 to node 6, we can


 22%|██▏       | 14/64 [00:18<01:08,  1.38s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 23%|██▎       | 15/64 [00:20<01:06,  1.36s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 25%|██▌       | 16/64 [00:21<01:05,  1.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 27%|██▋       | 17/64 [00:23<01:04,  1.38s/it]The attention mask and the pad token id were not set. As a consequence,

2 edges long. One possible path is: node 5 -> node 3 -> node 4. The weight of this path is the sum of the weights of its edges, which is 2


 64%|██████▍   | 41/64 [00:55<00:31,  1.35s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 66%|██████▌   | 42/64 [00:57<00:29,  1.36s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 67%|██████▋   | 43/64 [00:58<00:28,  1.36s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 69%|██████▉   | 44/64 [00:59<00:27,  1.37s/it]The attention mask and the pad token id were not set. As a consequence,

ZS Accuracy, shortest path 0.03125


In [16]:
for i in range(10):
    print(actual_labels[i], preds[i])

([4, 12, 10, 2], 12) ([4, 12, 3, 2], 5)
([11, 12, 4, 3, 9, 17, 7], 33) ([11, 12, 8, 7], 10)
([7, 1, 9, 3, 2], 12) ([7, 1, 2], 7)
([7, 8, 3], 3) ([7, 5, 3], 4)
([3, 5, 8], 4) ([3, 1, 5, 8], 7)
([2, 0, 4], 5) ([2, 1, 4], 3)
([11, 8, 9, 7, 0, 2], 31) ([8, 9, 7, 3, 6, 2], 22)
([1, 2, 3], 5) ([1, 2, 3], 2)
([0, 3, 4], 4) ([0, 1, 2, 4], 3)
([6, 1, 4], 3) ([6, 3, 4], 3)


In [17]:
train_dataset = NLGraph(root="./NLGraph", task="shortest_path", split="train", string_only=True)

In [18]:
context_examples = ""
for i in range(5):
    context_examples += train_dataset[i][0] + " " + train_dataset[i][2] + "\n"

print(context_examples)

In an undirected graph, the nodes are numbered from 0 to 4, and the edges are:
an edge between node 0 and node 1 with weight 4,
an edge between node 0 and node 3 with weight 4,
an edge between node 0 and node 2 with weight 3,
an edge between node 0 and node 4 with weight 1,
an edge between node 1 and node 3 with weight 3,
an edge between node 1 and node 4 with weight 1,
an edge between node 2 and node 3 with weight 3,
an edge between node 2 and node 4 with weight 2,
an edge between node 3 and node 4 with weight 1.
Q: Give the shortest path from node 1 to node 2.
A: The shortest path from node 1 to node 2 is 1,4,2 with a total weight of 3
In an undirected graph, the nodes are numbered from 0 to 16, and the edges are:
an edge between node 0 and node 3 with weight 4,
an edge between node 0 and node 13 with weight 7,
an edge between node 0 and node 14 with weight 8,
an edge between node 0 and node 6 with weight 5,
an edge between node 1 and node 16 with weight 5,
an edge between node 1 and

In [19]:
icl_accuracy = 0

for i in tqdm(range(len(test_dataset)), total=len(test_dataset)):

    graph_string = test_dataset[i][0]
    answer_string = test_dataset[i][2]

    nodes, edges = convert_string_to_graph(graph_string)
    path, weight = parse_answer(answer_string)

    input_prompt = context_examples + graph_string + " " + answer_string[: answer_string.find("is") + 3]
    inputs = base_tokenizer(input_prompt, return_tensors="pt").input_ids
    inputs = inputs.cuda()

    with torch.no_grad():
        
        # generate 20 tokens
        outputs = model.generate(inputs, max_length=inputs.shape[1] + 30, do_sample=False, temperature=0.0)
    
    # show decoded output
    decoded_output = base_tokenizer.decode(outputs[0][inputs.shape[1]:])
    # print("actual output", answer_string)
    # print("pred output", decoded_output)
    
    try:
        llm_path, llm_weight = parse_sp_output(decoded_output)
    except:
        llm_path, llm_weight = None, None
    
    # print("actual", path, weight)
    # print("pred", llm_path, llm_weight)

    if llm_path == None:
        print(decoded_output)
    else:
        # break
        if path == llm_path and weight == llm_weight:
            icl_accuracy += 1

    count += 1

    preds.append((llm_path, llm_weight))
    actual_labels.append((path, weight))
    if count == 10:
        break
    
print("ICL Accuracy, shortest path", icl_accuracy / count)

  0%|          | 0/64 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 1/64 [00:01<01:27,  1.40s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  3%|▎         | 2/64 [00:02<01:28,  1.42s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|▍         | 3/64 [00:04<01:25,  1.41s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

ICL Accuracy, shortest path 0.0078125
